In [1]:
# TENSOR FLOW AND KERAS

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import utils
import sklearn
import os
import cv2
import random


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Create training and validation sets

data_dir = "/drive/My Drive/pokeproj/PokemonData/"
data_dir = os.getcwd() + data_dir

image_size = (224, 224)
batch_size = 64

from tensorflow.keras.preprocessing.image import ImageDataGenerator

ImgDG = ImageDataGenerator(rescale=1./255, rotation_range=45, 
                    width_shift_range=.15, 
                    height_shift_range=.15, 
                    horizontal_flip=True, 
                    shear_range=.25,
                    zoom_range=.2,
                    validation_split=0.4,)

test_IDG = ImageDataGenerator(rescale=1./255,validation_split=0.4,)

train_ds = ImgDG.flow_from_directory(
    directory=data_dir,
    target_size=image_size,
    class_mode="categorical",   
    batch_size=batch_size,
    shuffle=True,
    seed=6969,
    subset="training",
    )

val_ds = test_IDG.flow_from_directory(
    directory=data_dir,
    target_size=image_size,
    class_mode="categorical",   
    batch_size=batch_size,
    shuffle=True,
    seed=6969,
    subset="validation",
)



Found 4125 images belonging to 149 classes.
Found 2654 images belonging to 149 classes.


In [3]:
# Build the Model

model = tf.keras.applications.MobileNet(input_shape=image_size+(3,), include_top=False, dropout=0.5)
model.trainable = False

model = tf.keras.Sequential([
  model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(149, activation='softmax')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 149)               152725    
Total params: 3,381,589
Trainable params: 152,725
Non-trainable params: 3,228,864
_________________________________________________________________


In [ ]:
# Fit the Model

epochs = 50

chpath = os.getcwd()+"/drive/My Drive/pokeproj/checkpts/"#"./checkptpath"
checkpt = keras.callbacks.ModelCheckpoint(filepath=chpath,
                                          monitor='val_accuracy',
                                          mode='max',
                                          save_best_only=True)
#callbacks = [ keras.callbacks.ModelCheckpoint("save-at-{epoch}.h5",)]
hist = model.fit(train_ds,
                  epochs=epochs,
                  callbacks=[checkpt],
                  validation_data=val_ds,
                  )

# steps_per_epoch = np.ceil(train_ds.n / batch_size)
# validation_steps = np.ceil(val_ds.n / batch_size)

# hist = model.fit(train_ds,
#                   steps_per_epoch = steps_per_epoch,
#                   epochs=epochs,
#                   callbacks=callbacks,
#                   validation_data=val_ds,
#                   validation_steps=validation_steps)


Epoch 1/50
65/65 [==============================] - ETA: 0s - loss: 5.3783 - accuracy: 0.0099WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/pokeproj/checkpts/assets
65/65 [==============================] - 108s 2s/step - loss: 5.3783 - accuracy: 0.0099 - val_loss: 5.1007 - val_accuracy: 0.0185
Epoch 2/50
62/65 [===========================>..] - ETA: 3s - loss: 4.9193 - accuracy: 0.0275

In [ ]:
# View training accuracy

plt.style.use('fivethirtyeight')
plt.figure(figsize=(14,14))
plt.plot(hist.history['val_accuracy'],label='accuracy',color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.yticks(np.arange(0, 1, step=0.04))
plt.show()

In [ ]:
# Sample Predictions

class_names = tf.keras.preprocessing.image_dataset_from_directory(data_dir,).class_names


corre_ct = 0
fig=plt.figure(figsize=(30,30))
sam_x,sam_y = next(val_ds) 
pred_ = model.predict(sam_x)

for i in range(10):
  pred,y = pred_[i].argmax(), sam_y[i].argmax()

  print("Pokemon is  ", class_names[pred], "| predicted pokemon: ", class_names[y])
  print("confidence %: ", max(pred_[i])) 


  res = class_names[pred]==class_names[y]
  if res:
    print("Correct")
    corre_ct += 1
  else:
    print("W R O N G")

  sub = fig.add_subplot(1, 10, i+1)
  sub.set_title(str(class_names[pred]) + ":" + str(res))
  plt.axis('off')
  plt.imshow(sam_x[i])


plt.show()
print("Number Correct: ", corre_ct,"/10")

In [ ]:
# dire = os.getcwd() + "/drive/My Drive/pokeproj/" + "mobn_149_poke_85.h5"
# model.save(dire)

# epoch 41 84.49%
# print(callbacks[0])
# test =model.load_weights(save-at-{41}.h5)